In [1]:
import time
import json
import bz2
import sys
from qwikidata.entity import WikidataItem
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.utils import dump_entities_to_json

In [2]:
from qwikidata.sparql import (get_subclasses_of_item,
                              return_sparql_query_results)

# send any sparql query to the wikidata query service and get full result back
sparql_query = """
# Living American politicians and their associated parties
select ?politician ?politicianLabel ?party ?partyLabel
where {
  # Get all senators
  ?politician wdt:P106 wd:Q82955; # is politician
           p:P102 ?partystatement; # And with a certain party
           wdt:P27 wd:Q30. # and is American
  minus { ?politician p:P570 ?deathDate .} # minus dead person 
#   OPTIONAL { ?politician skos:altLabel ?altLabel . FILTER (lang(?altLabel) = "en") } # add alternative labels, this will cause running time out

 

  
  ?partystatement ps:P102 ?party. # Get the party
  ?party wdt:P17 wd:Q30.          # it has to be a American party
  minus { ?partystatement pq:P582 ?partyEnd. } # but minus the ones is no longer a member of
  minus { ?party wdt:P361 ?partOf. }  # minus sub-group of other party like 'Minnesota Democratic–Farmer–Labor Party'
  minus { ?party wdt:P1416 ?affliation.} # minus affiliation of other party like "Republican Party of Minnesota"
         
  service wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
order by ?politicianLabel
"""
# link to the query https://w.wiki/4Lic
# return result in dict
res = return_sparql_query_results(sparql_query)


In [7]:
# dump in json
with open("../US_politician_party.json", "w") as out:
    json.dump(res, out, indent= 6)